In [2]:
import numpy as np
import visa
SENS_to_sens = [2e-9,5e-9,1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5,1e-4,2e-4,5e-4,1e-3,2e-3,5e-3,1e-2,2e-2,5e-2,1e-1,2e-1,5e-1,1]
OFLT_to_tc = [1e-5,3e-5,1e-4,3e-4,1e-3,3e-3,1e-2,3e-2,1e-1,3e-1,1,3,1e1,3e1,1e2,3e2,1e3,3e3,1e4,3e4]
def initsrs(instr):
    rm = visa.ResourceManager()
    rm.list_resources()
    return rm.open_resource(instr) 

# read X and Y from srs
def readsrs(srs):
    srs.query_ascii_values('SNAP?1,2')
    meas=srs.query_ascii_values('SNAP?1,2')
    print(meas)
    return meas
def auto_delay_read(srs,delay):
    srs.write('AGAN')
    time.sleep(delay)
    return readsrs(srs)
def fine_sweep_delay_read(srs,delay):
    time.sleep(delay)
    measR = abs(srs.query_ascii_values('OUTP?3')[0])
    sens_num = int(srs.query_ascii_values('SENS?')[0])
    sens_now=SENS_to_sens[sens_num]
    srs.query_ascii_values('OFLT?')
    tc=OFLT_to_tc[int(srs.query_ascii_values('OFLT?')[0])]
    print('tc', tc)
    meas=np.zeros(2)
    if SENS_to_sens[sens_num]<measR:
        #print(srs.query_ascii_values('OFLT?')[0])
        while(SENS_to_sens[sens_num]<measR and sens_num<len(SENS_to_sens)):
            sens_num = sens_num+1
            srs.write('SENS %d'%(sens_num))
  
            time.sleep(20*tc)
            measR=readsrs(srs)[0]
        meas=readsrs(srs)
    elif SENS_to_sens[sens_num-1]>measR:
        set_sens_num = sens_num
        while( set_sens_num>0 and SENS_to_sens[set_sens_num-1]>measR):
            set_sens_num= set_sens_num -1
        srs.write('SENS %d'%set_sens_num)
        time.sleep(15*tc)
        meas =readsrs(srs)
    else:
        meas=readsrs(srs)
    #print(meas)
    return meas

def smart_auto_delay_read(srs,delay):
    time.sleep(delay)
    measR = srs.query_ascii_values('OUTP?3')[0]
    sens_num = int(srs.query_ascii_values('SENS?')[0])
    sens_now=SENS_to_sens[sens_num]
    meas=np.zeros(2)
    if SENS_to_sens[sens_num]<measR:
        #print(srs.query_ascii_values('OFLT?')[0])
        srs.query_ascii_values('OFLT?')
        tc=OFLT_to_tc[int(srs.query_ascii_values('OFLT?')[0])]
        print('tc', tc)
        meas=auto_delay_read(srs,20*tc)
    elif SENS_to_sens[sens_num-1]>measR:
        set_sens_num = sens_num
        while( set_sens_num>0 and SENS_to_sens[set_sens_num-1]>measR):
            set_sens_num= set_sens_num -1
        srs.write('SENS %d'%set_sens_num)
    else:
        meas=readsrs(srs)
    #print(meas)
    return meas
      
def autosrs(srs):
    srs.write('ARSV')
    srs.write('AGAN')
    # can't do them the same time.
    #print(srs.query_ascii_values('STB?'))




#'SNAP i,j'# for 1X 2Y 3R 4 theta

In [6]:
OFLT_to_tc[13]

30.0